# Task 1: Light
---

In this notebook, we will study an *extremely* simple percolation model, which describes light rays travelling through a medium that contains **opaque** (light-absorbing) particles.
We would like to find out how **dense** the medium must be (how many opaque particles per unit area) for none of the light to percolate through.

Towards the end of the notebook, you will calculate a result analytically (i.e. by hand) and use this to verify that the model produces the correct results.
By performing this test, we will be able to move forward to simulating more complex systems, with the confidence that the model is performing as expected.

### Motivation

Why is this problem interesting?
The early universe was a very hot, dense **plasma** of charged particles (ionised Hydrogen and Helium, and electrons).
Being made up of charged particles, this plasma was opaque to light.
However, once the universe expanded cooled to a certain, very specific temperature, the ions and electrons combined to form neutral Hydrogen and Helium atoms, forming a **transparent** gas through which light could freely propagate.
This light can still be seen today; it is the [**Cosmic Microwave Background**](https://plancksatellite.org.uk/science/cmb/)!

# Using the model

### Importing the model code

The first thing we need to do is load, or **import** the code we're going to be using.

Run the cell below to do this.

In [ ]:
from lattice import SquareLattice
from model import PercolationModel

<div class="alert alert-block alert-warning">
<b>Warning!</b> 
    This cell must be run <b>every time you restart the kernel</b>.
    If you see errors that say either "name 'SquareLattice' is not defined" or "name 'PercolationModel' is not defined", you may need to run this cell again.
</div>

### Running an animation

Executing the cell below will generate an animation of the model, using the default parameters.

Animations take a little while to render, after which you can control their playback using the buttons below.

In [ ]:
lattice = SquareLattice()
model = PercolationModel(lattice)

model.animate()

You should see two boxes: the top one should be the animation, and the second is a static representation of the final state of the model.
The **live nodes** are coloured yellow, and the purple squares are **susceptible nodes** representing a uniform background medium.

### Adding parameters

We can change the behaviour of the model by changing its **parameters**.
The easiest way to do this is to pass **arguments** to `SquareLattice` and `PercolationModel`, which will need to be placed within the brackets `( ... )`.
Actually, we have already been passing `lattice` as an argument to `PercolationModel`!

Note that these terms closely resemble those used in mathematics.
For example, $\sin(x)$ is the result of the evaluating the *function* $\sin$ with the *argument* $x$.
In these notebooks, I will use the term 'function' in the loosest possible sense, as anything that is called with arguments using the syntax `function(arguments)`.
I may also conflate arguments and parameters occasionally!

In the cell below, we change the lattice size to make it taller than it is wide, and change the length of the animation so that it covers the entire evolution. Run the cell.

In [ ]:
lattice = SquareLattice(50, 10)
model = PercolationModel(lattice)

model.animate(51)

Next, we will add an argument to `PercolationModel`, after `lattice`, which assigns every node (square) a probability, $q$, of being **inert** at the start of the simulation.
In the simulation, the inert nodes look like grey squares.


<div class="alert alert-block alert-success">
    <b>Definition:</b> Each node has a fixed probability, $q$, of being inert for the duration of the simulation.
</div>

$$
\Pr(\text{node is inert}) \equiv q
$$

Run the cell below to see how adding the `q` parameter changes the behaviour of the simulation.
Try changing the line where `q` is defined.
To do this, change the number on the right hand side of the equals sign of the line `q = X`, and then run the cell again.

In [ ]:
q = 0.02  # change me!

lattice = SquareLattice(50, 10)
model = PercolationModel(lattice, q)

model.animate(51)

The eagle-eyed among you might notice that the number of rows in the simulation is actually one greater than the number you pass into `lattice`.
This is just because the model adds an extra row at the top, which is full of live nodes at time $t=0$.

### Optional further reading

If you've not seen Python code before and are interested in understanding what these lines of code are doing, feel free to take 5 minutes here to read over [Further reading: Python programming for the curious](#Further-reading:-Python-programming-for-the-curious) before returning to the task.

# Percolation

Run the cell below.

In [ ]:
q = 0.1

lattice = SquareLattice(100, 25)
model = PercolationModel(lattice, q)

model.animate(101)

You'll (almost certainly) see that the light gets absorbed well before it reaches the bottom edge.
We will describe this situation as a '**failure to percolate**'.


<div class="alert alert-block alert-success">
<b>Definition:</b> We will say that a simulation <b>percolates</b> if any square on the bottom edge of the box changes colour. In this case, this is interpreted as one or more light rays penetrating through the gas and reaching our detector at the bottom.
</div>

In the next cell, see if you can identify, by trial and error, the largest value of `q` that still allows the model to percolate.
Do not change the number of rows or columns; these should both be 50.

In [ ]:
q = 0.09  # change me!

lattice = SquareLattice(50, 50)
model = PercolationModel(lattice, q)

model.animate(51)

You should notice that, for values of `q` between about 0.05 and 0.1, the simulation sometimes percolates and sometimes doesn't.
This is because all nodes have a **probability** $q$ of being inert, but precisely which nodes are inert varies between simulations.

So, if you felt like something was wrong with the wording of the previous question, you were completely correct.
The question "*does the system percolate with* $q = X$?" is not a valid one.
Instead, we should ask "*what is the probability that the system percolates when* $q = X$?"

We will denote this probability $p$.

$$
\Pr(\text{simulation percolates}) \equiv p
$$

Note that $\Pr(A)$ is common notation for "the probability that $A$ occurs".

<div class="alert alert-block alert-warning">
<b>Warning!</b> 
    Before continuing, it is important to make sure you are completely happy with what the $q$ parameter represents, and what we mean when we say a simulation <b>percolates</b>, 
    If you're not sure, re-read the sections above, and the introductory notebook.
</div>

### A comment on random initial conditions

The origin of 'randomness' in our simulations comes from the configuration of inert nodes, not the actual model itself.
The technical way of describing the situation we have is a **deterministic model** with **random initial conditions**.

Such a setup might seem a bit strange, but is actually very well-motivated; it describes situations where we understand the physical laws governing the system very well, but we can't pinpoint a moment in time where we know *exactly* what the system looks like.
So, the idea is that we can run a lot of simulations with different initial conditions, to try and understand the system from a **statistical** point of view.

# Estimators and errors

We have discussed why we are interested in the **probability** that a simulation percolates ($p$) given the probability that any given node is inert ($q$), and why we are not so interested in whether or not a single, isolated simulation percolates.

However, it is important to realise that we can never directly calculate the percolation probability using simulations, but we can **estimate** it by running a large number of simulations and applying **statistics**.

<div class="alert alert-block alert-success">
    <b>Definition:</b> For a given value of $q$, we will take the fraction of simulations that percolate, $f$, to be an <b>estimator</b> for the true percolation probability, $p$.
</div>

It may seem like we are overly stressing this point, but the distinction between an **estimator** ($f$) and the **quantity that it is estimating** ($p$) is easily overlooked and yet absolutely crucial to get correct statistics.
The 3rd-year physics course *Fourier analysis and Statistics* goes into this (and much more) in more detail.

Recall that the Greek capital letter $\sum$ represents a sum: $\sum_{i=1}^N x_i = x_1 + x_2 + x_3 + \ldots + x_N$.

Given $N$ simulations, the fraction of these that percolate (i.e. our estimator) can be written

$$
f = \frac{1}{N} \sum_{i=1}^N b_i \tag{Estimator}
$$

where $b_i$ are equal to either 0 (for simulations which don't percolate) or 1 (for those that do), and $N$ is the number of simulations.
This formula should look familiar to those of you who have done statistics before; it is the **sample mean** of our simulations.

Now, there is something called the **Law of Large Numbers** which tells us that, as the size of the sample (i.e. number of simulations) tends to infinity then this estimator **converges** to the true value.
Hence,

$$
f \to p \qquad \text{as} \qquad N \to \infty \tag{Law of Large Numbers}
$$

However, this doesn't tell us *how quickly* the estimate converges.
What we really want to know is, *given $N$ measurements, how __confidently__ can we determine $p$ by calculating $f$?*

This is the role of the **standard error** on $f$, which we will denote $\delta_f$.
If we obtain an estimate $f$ based on $N$ measurements, then we can say with **approximately 68\% confidence** that the true value, $p$, lies in the interval $[f - \delta_f, f + \delta_f]$.

In general, the standard error on the sample mean is inversely proportional to the square root of the number of measurements, *provided the measurements are independent*.
This is true in our case; if one simulation succeeds in percolating, this has no effect on the probability of any other simulation percolating.

Because $f$ is the result of a number of independent trials ($b_1, b_2, \ldots, b_N$)  which can either succeed ($b_i=1$) or fail ($b_i=0$), it is governed by **Binomial statistics**.
Hence, the formula for the standard error is, strictly speaking, given by

$$
\delta_f^\text{true} = \sqrt{\frac{p (1 - p)}{N}} \, . \tag{Expected standard error}
$$

Since we just have an estimate for $p$, not $p$ itself, we must actually introduce **another estimator** for the standard error:

$$
\delta_f = \sqrt{\frac{f (1 - f)}{N-1}} \, . \tag{Standard error estimator}
$$

This formula may seem unfamiliar, but $\sqrt{\frac{N}{N-1} f (1 - f)}$ is actually the **standard deviation** $\sigma_b$ of our sample of simulation outcomes $b_1, b_2, \ldots, b_N$ (remember that each $b_i$ is either 1 or 0 depending on whether the simulation percolated or not).
Don't worry if this seems strange - just remember that this is a special case of the usual formula: $\delta_f = \sigma_b / \sqrt{N}$.

Don't worry too much about the fact that this second expression involves $N-1$ in the denominator instead of $N$.
However, this is a good opportunity to appreciate that statistics can be quite subtle!

<div class="alert alert-block alert-info">
<b>Lab book 1.1:</b>
    Imagine that we choose two values of $q$. One of them gives an estimate of $f \approx 0.1$, and the second gives $f \approx 0.5$.
    We want to achieve a standard error of $0.01$ on both of these estimates.
    Which estimate will require more simulations to be run to achieve $\delta_f = 0.01$?
    <b>[1]</b>
</div>

<div class="alert alert-block alert-info">
<b>Lab book 1.2:</b> 
    Imagine that we ran 10 simulations and none of them percolated, so that $f=0$.
    The formula above clearly shows that $\delta_f = 0$.
    Discuss how we should interpret this standard error, referring to equations written above.
    Is there really zero uncertainty in the value of $f$?
    What difference would it make if we run 1000 simulations and still measured $f=0$?
    <b>[2]</b>
</div>


Finally, note that we can apply the 68\% confidence rule to error bars on data points, meaning we should only expect approximately 2/3 of the data points to be within one standard error of the true value.
This tells us something very important: if we have correctly estimated the standard error, then we should **not** expect a line of best-fit to pass through **all** of the error bars.

Run the cell below to view another animation.

In [ ]:
q = 0.08

lattice = SquareLattice(50, 50)
model = PercolationModel(lattice, q)

model.animate(100)

In the cell below, $N$ = `N` simulations just like the one above are run (but no animation will be shown).
Based on these $N$ simulations, the cell will calculate and print the estimator $f$ and its standard error $\delta_f$.

By running the cell below with different values of `N`, find a value that consistently yields a standard error of $\delta_f \approx 0.02$.
Do not change the values of `q` or the lattice size - it should read `SquareLattice(50, 50)` and `PercolationModel(lattice, 0.08)`.

In [ ]:
N = 10  # change this number!
q = 0.08

lattice = SquareLattice(50, 50)
model = PercolationModel(lattice, q)

model.estimate_percolation_prob(N)

<div class="alert alert-block alert-info">
<b>Lab book 1.3:</b> 
    Make a note of this value of $N$ in your lab book. <b>[1]</b>
</div>

Remember, you can use the arrow keys to navigate between cells, which can be faster than clicking the cell)

The cell below contains exactly the same as the one above, but inside a **loop** that will run it 20 times, and print the values of $f$ and $\delta_f$ in two columns.

Set the `N` parameter to give a standard error of 0.02, and don't change `q` or the lattice size.

<div class="alert alert-block alert-info">
<b>Table 1:</b> 
    Run the cell below, recording the measurements of $f$ and $\delta_f$ in a table.
    Leave one additional column blank; we will use it later. <b>[3]</b>
</div>

In [ ]:
N = 10  # change this number!
q = 0.08

lattice = SquareLattice(50, 50)
model = PercolationModel(lattice, q)

model.loop_estimate_percolation_prob(N)

# Calculating the theoretical percolation probability

Our model of light rays percolating through a gas cloud may strike you as being extremely simple.
Indeed, it is not too tricky to calculate $p$, the probability that a simulation percolates, by hand.

Actually, we can use this to our advantage, since we will be able to check that our model is working as expected by comparing simulation results to analytical results.
These sorts of 'sanity checks' are very important in modelling; before making your model overly complicated, it is very sensible to check that it reproduces a known result in a simple test-case!

Before doing the calculation let us define our variables.
As a reminder, each node has a fixed probability of being inert, 

$$
\Pr(\text{node is inert}) \equiv q
$$

The lattice has $n$ rows and $m$ columns, not counting the top row which starts full of live nodes at time $t=0$.

The first thing to notice is that, since the light in our simulation can only travel vertically, each column is independent of every other.

<div class="alert alert-block alert-info">
<b>Lab book 1.4a:</b> 
    Show that the probability $c$ that a <b>single column</b> percolates (i.e. there are no inert nodes in that column) is

$$\Pr(\text{single col percolates}) \equiv c = (1 - q)^n$$
    <b>[1]</b>
</div>

<div class="alert alert-block alert-info">
<b>Lab book 1.4b:</b> 
    Next, show that the probability that <b>at least one</b> of the $m$ columns percolates is
    
$$\Pr(\text{at least one col percolates}) \equiv p = 1 - (1 - c)^m$$
    <b>[1]</b>
</div>

**Hint 1**: If $\Pr(A)$ describes the probability of event $A$ occuring and $\Pr(B)$ describes the probability of another, independent event occurring, then the probability that *both* $A$ *and* $B$ occur is 

$$
\Pr(\text{both } A \text{ and } B \text{ occur}) \equiv
\Pr(A \cap B) = \Pr(A) \Pr(B)
$$

**Hint 2**: The probability of *at least one* column percolates is one minus the probability that *zero* columns percolate.


## Sanity checking the model

Thanks to your mathematical prowess, we are now in the very privileged position of being able to test our simulation results against the correct answers.

Use the formula you have just derived to calculate $p$ for the case of $q = 0.08$ and $n = m = 50$.

Then, calculate the expected standard error $\delta_f^\text{true}$, and check that the values in your table are in rough agreement.
Note that what you are checking here is referred to as *the error on the error*!

<div class="alert alert-block alert-info">
<b>Lab book 1.5:</b> 
    Record your calculations of $p$ and $\delta_f^\text{true}$ in your lab book.
    <b>[1]</b>
</div>

**Hint:** Remember, you will have to use your chosen value for the number of trials, $N$, when calculating the expected standard error using the formula provided earlier; the expected error does not depend on the actual outcomes of your simulations, but it does depend on the *number of data points* used to calculate the mean.


In your table, you should have one column remaining.
In this column, you should calculate and write down the numerical answer to

$$
\Big| \frac{f - p}{\delta_f} \Big|
$$

which is the *number of standard errors separating $f$ and $p$*.

<div class="alert alert-block alert-info">
<b>Table 1 (part 2):</b> 
    Fill the final column of your table as instructed above. <b>[1]</b>
</div>

<div class="alert alert-block alert-info">
<b>Lab book 1.6:</b> 
    How many of the 20 estimates are within $1\delta_f$ of the correct answer?
    Does this match your expectations?
    <b>[1]</b>
</div>

# Standard deviations

The general formula for the (sample) **standard deviation** is

$$
\sigma_x = \sqrt{\frac{\sum_{i=1}^N(x_i - \bar{x})^2}{N-1}} \tag{Standard deviation - general formula}
$$

where $x_i$ are the measurements, $N$ is the number of measurements taken, and $\bar{x} = \frac{1}{N} \sum_{i=1}^N x_i$ is the mean value.

We also mentioned that the standard deviation of a sample of $N$ simulation outcomes is

$$
\sigma_b = \sqrt{\frac{N}{N-1} f (1 - f)} \tag{Standard deviation - simulations}
$$

where

$$
f = \frac{1}{N} \sum_{i=1}^N b_i = \bar{b}
$$

and each simulation outcome $b_i$ is either a 1 or a 0, depending on whether the simulation percolated or not.

<div class="alert alert-block alert-info">
<b>Lab book 1.7:</b> 
    Use the general formula for the standard deviation to show that the standard deviation of a sample of $N$ simulations is given by the formula for $\sigma_b$ above. I.e. substitute $x$ for $b$ in the formula for $\sigma_x$. <b>[2]</b>
</div>

**Hint 1:** Note that, for a constant $a$, we have $\sum_{i=1}^N a = a + a + a + \ldots (N \text{ times}) = Na$ .

**Hint 2:** $\sum_{i=1}^N a x_i = a \sum_{i=1}^N x_i$  for a constant $a$.

<div class="alert alert-block alert-info">
<b>Lab book 1.8:</b> 
    Use the general formula for the standard deviation to calculate the standard deviation $\sigma_f$ of your 20 estimates of $f$ from your table.
    Comment on how your result compares with the standard error $\delta_f$. (Bonus) Can you explain this? <b>[2]</b>
</div>

## The percolation transition

<div class="alert alert-block alert-success">
    <b>Definition:</b> The <b>percolation transition</b> refers to the transition between almost total certainty that the model will percolate ($p \approx 1$), and almost complete certainty that it will not ($p \approx 0$).
    This sort of relatively sudden jump between two <b>qualitatively different</b> behaviours in a system is known as a <b>phase transition</b>.
</div>

The cell below contains a function which, just as before, runs a number (in this case $N = 50$) of simulations and calculates the fraction that percolate.
However, this function also *loops* over many (50) values of `q` and plots the results along with the theoretical prediction.
We are referring to this as a **parameter scan** over the $q$ parameter.

The difference with the true probability $f - p$ is also plotted.
We will refer to these as **residuals**, although usually the residuals are the difference between data points and the line of best fit.

In the cell, notice that we have set values for the following parameters:
* `qmin` : the lowest value of `q`
* `qmax` : the highest value of `q`


Run the cell.

In [ ]:
from parameter_scan import parameter_scan

lattice = SquareLattice(50, 50)
model = PercolationModel(lattice)

qmin = 0.03
qmax = 0.16
N = 50

parameter_scan(model, qmin, qmax, N)

<div class="alert alert-block alert-info">
<b>Lab book 1.9:</b>
    Some of the error bars may not intersect with the theoretical curve. Does that imply that
    
    (a) the measurement is wrong and we should re-take it.
    (b) the theory is wrong.
    (c) neither (a) nor (b).
    
Explain your answer. <b>[1]</b>
</div>

<div class="alert alert-block alert-danger">
<b>Notebook complete:</b> You're ready to move on to Notebook 2.
</div>

# Further reading: Python programming for the curious


However, this is purely to provide some explanation for those who want it.
It is **absolutely not part of of the laboratory to learn this terminology**, and to be totally clear, **no programming knowledge is required to complete this laboratory and gain full marks**.

<div class="alert alert-block alert-success">
    <b>Definition:</b> <b>Modules</b> are essentially just locations (files) where bits of code are grouped together and stored.
    
We can load, or *import* some code from a module using the following syntax
```python
from my_module import my_code
```
</div>

In the first cell we import two 'bits of code', `SquareLattice` and `PercolationModel`, from two different modules, `lattice` and `model`.

```python
from lattice import SquareLattice
from model import PercolationModel
```

So what are these 'bits of code' that we've imported?
In this case, `SquareLattice` and `PercolationModel` are both *classes*.

<div class="alert alert-block alert-success">
    <b>Definition:</b> <b>Classes</b> act as flexible templates, from which Python is able to create <b>objects</b>.
    The flexibility of classes comes from the fact that we can specify certain <b>parameters</b> (like tuning the dials on a machine) when creating an object.
The syntax for creating an object from a class is
    
```python
my_object = MyClass(my_parameter, my_second_parameter, ...)
```
</div>

The next thing we do is create two objects: `lattice` and `model` from the two classes.

```python
lattice = SquareLattice()
model = PercolationModel(lattice)
```
`lattice` is given as a parameter (or argument) to `PercolationModel`.
You might think it looks like there are no other parameters.
There are, but by not specifying them we just use the default values.

Once an object is created, we can interact with it.
In particular, we might want to ask it to
* Tell us the value of a certain *attribute* : `print(object.attribute)`
* Execute one of its *methods* : `object.method(argument_1, argument_2, ...)`
* Modify the value of an attribute : `object.attribute = new_value`

So when we run the animation,
```python
model.animate(n_steps=100)
```
we're telling `model` to execute its `animation` method, while passing the *argument* `n_steps=100` that tells it to complete 100 iterations.


Now this all sounds incredibly abstract, but underneath the strange terminology are concepts that will seem very intuitive.
A 'real world' example will serve us well to explain how classes and objects work.

### A 'real world' example
---

Imagine you are the chief baker in a bakery.
Every day you make a selection of delicious loaves, cakes, pastries from scratch.
However, your bakery has become so popular that you just can't keep up with demand, so you decide to invest in a machine that can make the dough and the batter for you.

In this story, the machine will be the analogue of the Python programming language.
It contains lots of complicated circuitry (*microcode*) but as the baker you rarely need to worry about all that.

The machine has two main settings: 'batter' (used to make cakes) and 'dough' (for bread).
These are two distinct *classes* of things that the machine can make.

Of course, not all dough is the same, and your desired recipe will depend on what kind of bread you're making.
So, after selecting the 'dough' setting, you are presented with a list of *parameters* - type of flour, amount of yeast, amount of salt... - which you can tune to get exactly the dough that you want.

You set the parameters to how you want them, and hit the 'start' button.
A couple of hours later, the dough pops out.
We now have an *object* that has been created based on the 'dough' class and the specific parameters you chose when starting the machine.

What's next?
You might want to leave the dough to prove for a while, perhaps shape it into a nice loaf-shape, and at some point it's going to need baking.
All of these things are analogous to *methods* belonging to an object.
You could also say that the dough has certain *attributes*, for example: 'time spent proving', 'shape', 'baked'.

Let's see how this story would play out in some Python code!
```python
from bread_maker import Dough, Batter  # import our classes

# Make the brown_dough object from the Dough class, using these specific parameters
brown_dough = Dough(water="225ml", flour="300g", flour_type="wholemeal", yeast="7g", salt="1tsp")

if brown_dough.shape is not "pretty":
    brown_dough.make_pretty()  # shape the dough into a nice round shape

brown_dough.prove(hours=2)  # prove the dough

brown_dough.bake(hours=1)  # bake the dough

brown_dough.is_it_baked  # prints "True" since it has been baked

brown_dough.set_price("£1.50")  # set an attribute that is the price of the bread
```